In [1]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [7]:
# Initialize the folders with train, test and validation datasets (in "/My Drive/..." or from your local repository where you have downloaded data):

train = './train/train'
val =   './val/val'
test =  './test/test'

# The shape of the RGB image
img_width, img_height, channels = 150, 150, 3 # you can try different sizes

# input shape
input_shape = (img_width, img_height, 3)
# position matters!
# Number_of_channels can be at the first or the last position
# in our case - "channels last"

# minibatch size
batch_size = 64
# train set size
nb_train_samples = 20000
# validation set size 
nb_validation_samples = 2490
# test set size
nb_test_samples = 2500

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

val_generator = datagen.flow_from_directory(
    val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 2490 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [42]:
model = Sequential()

# 1: +Convolutional
# For example:
model.add(Conv2D(16, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))

# 2: +Pooling
model.add(MaxPooling2D((2,2)))
# 3:
model.add(Conv2D(32, (3, 3)))
#     +Relu
model.add(Activation('relu'))
# 4:  +Pooling 
model.add(MaxPooling2D((2,2)))
# 5:  +Convolutional
model.add(Conv2D(64, (3, 3)))
#     +Relu
model.add(Activation('relu'))
# 6:  +Pooling 
model.add(MaxPooling2D((2,2)))
# 7:  +Flattening
model.add(Flatten())
# 8:  +Dense
model.add(Dense(64))
#     +ReLu
model.add(Activation('relu'))
# 9:  +Dropout
model.add (Dropout (0.5))
# 10: +Dense
model.add(Dense(1))
#     +Sigmoid
model.add(Activation('sigmoid'))


In [43]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
# use generator for training the model ("fit" method analog)
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

<ipython-input-44-b3ea65b66c88>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
312/312 [==============================] - 142s 453ms/step - loss: 0.5952 - accuracy: 0.6725 - val_loss: 0.5441 - val_accuracy: 0.7253
Epoch 2/10
312/312 [==============================] - 131s 419ms/step - loss: 0.5147 - accuracy: 0.7477 - val_loss: 0.4788 - val_accuracy: 0.7743
Epoch 3/10
312/312 [==============================] - 129s 414ms/step - loss: 0.4556 - accuracy: 0.7897 - val_loss: 0.4352 - val_accuracy: 0.8051
Epoch 4/10
312/312 [==============================] - 126s 405ms/step - loss: 0.4062 - accuracy: 0.8173 - val_loss: 0.4060 - val_accuracy: 0.8096
Epoch 5/10
312/312 [==============================] - 122s 391ms/step - loss: 0.3684 - accuracy: 0.8347 - val_loss: 0.3939 - val_accuracy: 0.8215
Epoch 6/10
312/312 [==============================] - 123s 395ms/step - loss: 0.3210 - accuracy: 0.8617 - val_loss: 0.3842 - val_accuracy: 0.8302
Epoch 7/10
312/312 [==============================] - 122s 391ms/step - loss: 0.2896 - accuracy: 0.8739 - val_loss: 0.3889 -

In [47]:
# NOTE: if the accuracy on test data after 15 epochs is less than 80% smth goes wrong

scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

<ipython-input-47-f790ba523922>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


Accuracy on test data: 84.74%


### Going deeper

In [3]:
# First, download the weights of the VGG16 network trained on the ImageNet dataset:

vgg16_net = VGG16(weights='imagenet', 
                  include_top=False,      # we take only the "convolution" part, the last layers we add ourselves
                  input_shape=(150, 150, 3))
vgg16_net.trainable = False               # clearly prescribe that we do NOT overload the network.
                                          # Weights VGG16 in the process of learning will remain unchanged!

vgg16_net.summary()                       # pay attention to the number of trained and untrained parameters

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [49]:
# add layers to VGG16:

model = Sequential()
model.add(vgg16_net)

# + flattening
model.add(Flatten())
# + Dense fullyconnected layer with 256 neurons
model.add(Dense(256))
# + ReLu
model.add(Activation('relu'))
# + Dropout
model.add (Dropout (0.5))
# + Dense layer with 1 neuron
model.add(Dense(1))
# + sigmoid
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 8192)              0         
                                                                 
 dense_5 (Dense)             (None, 256)               2097408   
                                                                 
 activation_24 (Activation)  (None, 256)               0         
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
 activation_25 (Activation)  (None, 1)                

In [53]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

C:\Users\vinni\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [54]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/5


<ipython-input-54-92c8f2b4dc55>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


312/312 [==============================] - 957s 3s/step - loss: 0.5110 - accuracy: 0.7385 - val_loss: 0.3468 - val_accuracy: 0.8643
Epoch 2/5
312/312 [==============================] - 942s 3s/step - loss: 0.3380 - accuracy: 0.8521 - val_loss: 0.2954 - val_accuracy: 0.8820
Epoch 3/5
312/312 [==============================] - 940s 3s/step - loss: 0.2965 - accuracy: 0.8735 - val_loss: 0.2721 - val_accuracy: 0.8943
Epoch 4/5
312/312 [==============================] - 931s 3s/step - loss: 0.2728 - accuracy: 0.8846 - val_loss: 0.2553 - val_accuracy: 0.9013
Epoch 5/5
312/312 [==============================] - 884s 3s/step - loss: 0.2583 - accuracy: 0.8906 - val_loss: 0.2493 - val_accuracy: 0.9038


In [55]:
import tensorflow as tf

In [58]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)

[]


In [59]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

<ipython-input-59-dca356712a64>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


Accuracy on test data: 89.14%


### Going deeper with VG19

In [4]:
# First, download the weights of the VGG16 network trained on the ImageNet dataset:

vgg19_net = VGG19(weights='imagenet', 
                  include_top=False,      # we take only the "convolution" part, the last layers we add ourselves
                  input_shape=(150, 150, 3))
vgg19_net.trainable = False               # clearly prescribe that we do NOT overload the network.
                                          # Weights VGG16 in the process of learning will remain unchanged!

vgg19_net.summary()                       # pay attention to the number of trained and untrained parameters

80150528/80134624 [==============================] - 7s 0us/step
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

In [5]:
# add layers to VGG16:

model = Sequential()
model.add(vgg19_net)

# + flattening
model.add(Flatten())
# + Dense fullyconnected layer with 256 neurons
model.add(Dense(256))
# + ReLu
model.add(Activation('relu'))
# + Dropout
model.add (Dropout (0.5))
# + Dense layer with 1 neuron
model.add(Dense(1))
# + sigmoid
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
 activation_1 (Activation)   (None, 1)                

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

<ipython-input-11-92c8f2b4dc55>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
312/312 [==============================] - 1230s 4s/step - loss: 0.3785 - accuracy: 0.8396 - val_loss: 0.2579 - val_accuracy: 0.8914
Epoch 2/5
312/312 [==============================] - 1125s 4s/step - loss: 0.2647 - accuracy: 0.8865 - val_loss: 0.2505 - val_accuracy: 0.8976
Epoch 3/5
312/312 [==============================] - 1219s 4s/step - loss: 0.2455 - accuracy: 0.8950 - val_loss: 0.2554 - val_accuracy: 0.8910
Epoch 4/5
312/312 [==============================] - 1206s 4s/step - loss: 0.2301 - accuracy: 0.9037 - val_loss: 0.2526 - val_accuracy: 0.9005
Epoch 5/5
312/312 [==============================] - 1134s 4s/step - loss: 0.2178 - accuracy: 0.9062 - val_loss: 0.2448 - val_accuracy: 0.9001


In [12]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

<ipython-input-12-dca356712a64>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


Accuracy on test data: 89.30%
